In [1]:
from __future__ import print_function
import IPython
import sys
from music21 import *
import numpy as np
from grammar import *
from qa import *
from preprocess import *
from keras.models import load_model, Model
from keras.layers import Dense, Activation, Dropout, Input, LSTM, Reshape, Lambda, RepeatVector
from keras.initializers import glorot_uniform
from keras.utils import to_categorical
from keras.optimizers import Adam
from keras import backend as K

Using TensorFlow backend.


# Process Music Data

Value as a note, which comprises a pitch and a duration. For example, if you press down a specific piano key for 0.5 seconds, then you have just played a note. 

In [99]:
#"R,0.125" : a rest element of  (1/32) length, or 1/8 quarter note. 
#"C,0.125<M-2,m-6>" : chord note of (1/32) length, generated
#                             anywhere from minor 6th down to major 2nd down.

In [2]:
# get data
chords, abstract_grammars = get_musical_data('./midi/original_metheny.mid')
corpus, values, val_indices, indices_val = get_corpus_data(abstract_grammars)
print('corpus length:', len(corpus))
print('total # of values:', len(values))

corpus length: 193
total # of values: 78


In [5]:
chords[1][0].offset

476.0

In [6]:
chords[1][1].offset

478.0

In [7]:
chords[2][0].offset

480.0

In [8]:
abstract_grammars

['C,0.500 C,0.333,<P1,d-5> C,0.250,<M-2,m-6> C,0.250,<M3,d-3> C,0.250,<d6,m2> C,0.250,<d5,P1> C,0.667,<M2,d-4> C,0.250,<d1,P-5> C,0.250,<P-4,d-8> S,0.250,<d4,M-2> S,0.250,<P5,A1> C,0.250,<d5,P1>',
 'C,0.500 S,0.250,<m2,P-4> C,0.250,<P4,m-2> A,0.250,<P4,m-2> S,0.500,<d1,P-5> C,0.250,<P1,d-5> C,0.250,<m2,P-4> A,0.250,<m2,P-4> C,0.250,<M2,d-4> A,0.250,<d4,M-2> C,0.250,<P4,m-2> C,0.250,<P4,m-2>',
 'C,0.250 C,0.250,<d2,A-4> A,0.250,<m-2,d-6> C,0.250,<d3,M-3> A,0.250,<M3,d-3> C,0.250,<d6,m2> C,0.250,<A4,d-2> C,0.250,<d3,M-3> C,0.250,<m2,P-4> A,0.250,<m2,P-4> S,0.250,<P1,d-5> C,0.250,<m2,P-4> C,0.250,<P1,d-5> C,0.250,<m2,P-4> A,0.250,<m2,P-4>',
 'C,0.250 S,0.250,<d6,m2> X,0.250,<A4,d-2> S,0.250,<dd5,d1> C,0.250,<P4,m-2> C,0.250,<M3,d-3> C,0.250,<dd5,d1> S,0.250,<P4,m-2> C,0.250,<P4,m-2> C,0.250,<M3,d-3> C,0.250,<dd5,d1> A,0.250,<M3,d-3>',
 'C,0.500 A,0.333,<M2,d-4> C,0.250,<d4,M-2> A,0.333,<M2,d-4> C,0.250,<m2,P-4> C,0.250,<A-4,P-8> C,0.250,<P4,m-2> A,0.250,<P4,m-2> S,0.250,<d5,P1> A,0.333,<P

In [9]:
corpus

['C,0.500',
 'C,0.333,<P1,d-5>',
 'C,0.250,<M-2,m-6>',
 'C,0.250,<M3,d-3>',
 'C,0.250,<d6,m2>',
 'C,0.250,<d5,P1>',
 'C,0.667,<M2,d-4>',
 'C,0.250,<d1,P-5>',
 'C,0.250,<P-4,d-8>',
 'S,0.250,<d4,M-2>',
 'S,0.250,<P5,A1>',
 'C,0.250,<d5,P1>',
 'C,0.500',
 'S,0.250,<m2,P-4>',
 'C,0.250,<P4,m-2>',
 'A,0.250,<P4,m-2>',
 'S,0.500,<d1,P-5>',
 'C,0.250,<P1,d-5>',
 'C,0.250,<m2,P-4>',
 'A,0.250,<m2,P-4>',
 'C,0.250,<M2,d-4>',
 'A,0.250,<d4,M-2>',
 'C,0.250,<P4,m-2>',
 'C,0.250,<P4,m-2>',
 'C,0.250',
 'C,0.250,<d2,A-4>',
 'A,0.250,<m-2,d-6>',
 'C,0.250,<d3,M-3>',
 'A,0.250,<M3,d-3>',
 'C,0.250,<d6,m2>',
 'C,0.250,<A4,d-2>',
 'C,0.250,<d3,M-3>',
 'C,0.250,<m2,P-4>',
 'A,0.250,<m2,P-4>',
 'S,0.250,<P1,d-5>',
 'C,0.250,<m2,P-4>',
 'C,0.250,<P1,d-5>',
 'C,0.250,<m2,P-4>',
 'A,0.250,<m2,P-4>',
 'C,0.250',
 'S,0.250,<d6,m2>',
 'X,0.250,<A4,d-2>',
 'S,0.250,<dd5,d1>',
 'C,0.250,<P4,m-2>',
 'C,0.250,<M3,d-3>',
 'C,0.250,<dd5,d1>',
 'S,0.250,<P4,m-2>',
 'C,0.250,<P4,m-2>',
 'C,0.250,<M3,d-3>',
 'C,0.250,

In [10]:
values

{'A,0.250,<M2,d-4>',
 'A,0.250,<M3,d-3>',
 'A,0.250,<P-4,d-8>',
 'A,0.250,<P1,d-5>',
 'A,0.250,<P4,m-2>',
 'A,0.250,<d4,M-2>',
 'A,0.250,<d5,P1>',
 'A,0.250,<m-2,d-6>',
 'A,0.250,<m2,P-4>',
 'A,0.333,<A4,d-2>',
 'A,0.333,<M2,d-4>',
 'A,0.333,<P1,d-5>',
 'C,0.250',
 'C,0.250,<A-4,P-8>',
 'C,0.250,<A4,d-2>',
 'C,0.250,<M-2,m-6>',
 'C,0.250,<M2,d-4>',
 'C,0.250,<M3,d-3>',
 'C,0.250,<P-4,d-8>',
 'C,0.250,<P1,d-5>',
 'C,0.250,<P11,M7>',
 'C,0.250,<P4,m-2>',
 'C,0.250,<P5,A1>',
 'C,0.250,<d1,P-5>',
 'C,0.250,<d2,A-4>',
 'C,0.250,<d3,M-3>',
 'C,0.250,<d4,M-2>',
 'C,0.250,<d5,P1>',
 'C,0.250,<d6,m2>',
 'C,0.250,<dd5,d1>',
 'C,0.250,<m-2,d-6>',
 'C,0.250,<m2,P-4>',
 'C,0.250,<m3,m-3>',
 'C,0.250,<m7,M3>',
 'C,0.333,<A4,d-2>',
 'C,0.333,<M2,d-4>',
 'C,0.333,<P1,d-5>',
 'C,0.333,<P5,A1>',
 'C,0.333,<d1,P-5>',
 'C,0.333,<m2,P-4>',
 'C,0.333,<m3,m-3>',
 'C,0.333,<m7,M3>',
 'C,0.500',
 'C,0.500,<P4,m-2>',
 'C,0.500,<m2,P-4>',
 'C,0.500,<m6,M2>',
 'C,0.667,<M2,d-4>',
 'C,0.667,<d6,m2>',
 'C,0.750,<m3

In [11]:
val_indices

{'A,0.250,<M2,d-4>': 5,
 'A,0.250,<M3,d-3>': 58,
 'A,0.250,<P-4,d-8>': 19,
 'A,0.250,<P1,d-5>': 17,
 'A,0.250,<P4,m-2>': 55,
 'A,0.250,<d4,M-2>': 30,
 'A,0.250,<d5,P1>': 72,
 'A,0.250,<m-2,d-6>': 25,
 'A,0.250,<m2,P-4>': 75,
 'A,0.333,<A4,d-2>': 22,
 'A,0.333,<M2,d-4>': 27,
 'A,0.333,<P1,d-5>': 57,
 'C,0.250': 48,
 'C,0.250,<A-4,P-8>': 38,
 'C,0.250,<A4,d-2>': 21,
 'C,0.250,<M-2,m-6>': 39,
 'C,0.250,<M2,d-4>': 32,
 'C,0.250,<M3,d-3>': 16,
 'C,0.250,<P-4,d-8>': 77,
 'C,0.250,<P1,d-5>': 65,
 'C,0.250,<P11,M7>': 52,
 'C,0.250,<P4,m-2>': 40,
 'C,0.250,<P5,A1>': 37,
 'C,0.250,<d1,P-5>': 74,
 'C,0.250,<d2,A-4>': 15,
 'C,0.250,<d3,M-3>': 69,
 'C,0.250,<d4,M-2>': 31,
 'C,0.250,<d5,P1>': 54,
 'C,0.250,<d6,m2>': 13,
 'C,0.250,<dd5,d1>': 14,
 'C,0.250,<m-2,d-6>': 20,
 'C,0.250,<m2,P-4>': 28,
 'C,0.250,<m3,m-3>': 51,
 'C,0.250,<m7,M3>': 46,
 'C,0.333,<A4,d-2>': 44,
 'C,0.333,<M2,d-4>': 43,
 'C,0.333,<P1,d-5>': 12,
 'C,0.333,<P5,A1>': 10,
 'C,0.333,<d1,P-5>': 8,
 'C,0.333,<m2,P-4>': 56,
 'C,0.333,<

In [12]:
indices_val

{0: 'S,0.333,<m2,P-4>',
 1: 'C,0.333,<m3,m-3>',
 2: 'S,0.250,<d2,A-4>',
 3: 'X,0.250,<M-2,m-6>',
 4: 'S,0.250,<P5,A1>',
 5: 'A,0.250,<M2,d-4>',
 6: 'X,0.250,<A4,d-2>',
 7: 'S,0.250,<m3,m-3>',
 8: 'C,0.333,<d1,P-5>',
 9: 'S,0.250,<P1,d-5>',
 10: 'C,0.333,<P5,A1>',
 11: 'S,0.667,<m3,m-3>',
 12: 'C,0.333,<P1,d-5>',
 13: 'C,0.250,<d6,m2>',
 14: 'C,0.250,<dd5,d1>',
 15: 'C,0.250,<d2,A-4>',
 16: 'C,0.250,<M3,d-3>',
 17: 'A,0.250,<P1,d-5>',
 18: 'C,0.500,<m6,M2>',
 19: 'A,0.250,<P-4,d-8>',
 20: 'C,0.250,<m-2,d-6>',
 21: 'C,0.250,<A4,d-2>',
 22: 'A,0.333,<A4,d-2>',
 23: 'S,0.667,<d5,P1>',
 24: 'X,0.250,<m6,M2>',
 25: 'A,0.250,<m-2,d-6>',
 26: 'S,0.250,<d1,P-5>',
 27: 'A,0.333,<M2,d-4>',
 28: 'C,0.250,<m2,P-4>',
 29: 'C,0.667,<d6,m2>',
 30: 'A,0.250,<d4,M-2>',
 31: 'C,0.250,<d4,M-2>',
 32: 'C,0.250,<M2,d-4>',
 33: 'S,0.250,<P4,m-2>',
 34: 'C,0.500,<P4,m-2>',
 35: 'S,0.250,<d4,M-2>',
 36: 'S,0.250,<dd5,d1>',
 37: 'C,0.250,<P5,A1>',
 38: 'C,0.250,<A-4,P-8>',
 39: 'C,0.250,<M-2,m-6>',
 40: 'C,0.25

In [14]:
# number of different values or words in corpus
N_values = len(set(corpus))

# Create Sentences

In [13]:
# cut the corpus into semi-redundant sequences of max_len values
max_len = 20
sentences = []
next_values = []
step = 3

for i in range(0, len(corpus) - max_len, step):
    sentences.append(corpus[i: i + max_len]) #[i, i+max_len)
    next_values.append(corpus[i + max_len])
print('nb sequences:', len(sentences))

nb sequences: 58


# Transform data into binary matrices

In [15]:
X = np.zeros((len(sentences), max_len, N_values), dtype=np.bool)
y = np.zeros((len(sentences), N_values), dtype=np.bool)

In [16]:
for i, sentence in enumerate(sentences):
    for t, val in enumerate(sentence):
        X[i, t, val_indices[val]] = 1
    y[i, val_indices[next_values[i]]] = 1

In [17]:
X.shape

(58, 20, 78)

# LSTM Models

In [18]:
from __future__ import print_function

from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
import numpy as np

In [19]:
N_epochs=128
# build a 2 stacked LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(max_len, N_values)))
model.add(Dropout(0.2))
model.add(LSTM(128, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(N_values))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='rmsprop')
model.fit(X, y, batch_size=128, nb_epoch=N_epochs)

/Users/yfu/anaconda/lib/python3.6/site-packages/keras/models.py:848: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 1/128
58/58 [==============================] - 1s - loss: 4.3581
Epoch 2/128
58/58 [==============================] - 0s - loss: 4.3242
Epoch 3/128
58/58 [==============================] - 0s - loss: 4.2776
Epoch 4/128
58/58 [==============================] - 0s - loss: 4.1344
Epoch 5/128
58/58 [==============================] - 0s - loss: 3.7913
Epoch 6/128
58/58 [==============================] - 0s - loss: 3.7616
Epoch 7/128
58/58 [==============================] - 0s - loss: 3.7228
Epoch 8/128
58/58 [==============================] - 0s - loss: 3.5981
Epoch 9/128
58/58 [==============================] - 0s - loss: 3.5380
Epoch 10/128
58/58 [==============================] - 0s - loss: 3.4847
Epoch 11/128
58/58 [==============================] - 0s - loss: 3.5321
Epoch 12/128
58/58 [==============================] - 0s - loss: 3.3918
Epoch 13/128
58/58 [==============================] - 0s - loss: 3.3768
Epoch 14/128
58/58 [==============================] - 0s - loss: 3.3685
E

58/58 [==============================] - 0s - loss: 0.7148
Epoch 115/128
58/58 [==============================] - 0s - loss: 0.6380
Epoch 116/128
58/58 [==============================] - 0s - loss: 0.4150
Epoch 117/128
58/58 [==============================] - 0s - loss: 0.3946
Epoch 118/128
58/58 [==============================] - 0s - loss: 0.3489
Epoch 119/128
58/58 [==============================] - 0s - loss: 0.3522
Epoch 120/128
58/58 [==============================] - 0s - loss: 0.3296
Epoch 121/128
58/58 [==============================] - 0s - loss: 0.3537
Epoch 122/128
58/58 [==============================] - 0s - loss: 0.3374
Epoch 123/128
58/58 [==============================] - 0s - loss: 0.2571
Epoch 124/128
58/58 [==============================] - 0s - loss: 0.2838
Epoch 125/128
58/58 [==============================] - 0s - loss: 0.3036
Epoch 126/128
58/58 [==============================] - 0s - loss: 0.3732
Epoch 127/128
58/58 [==============================] - 0s - loss:

# Predict

In [20]:
def __sample(a, temperature=1.0):
    a = np.log(a) / temperature
    a = np.exp(a) / np.sum(np.exp(a))
    return np.argmax(np.random.multinomial(1, a, 1))

In [21]:
def __predict(model, x, indices_val, diversity):
    preds = model.predict(x, verbose=0)[0]
    next_index = __sample(preds, diversity)
    next_val = indices_val[next_index]
    
    return next_val

In [22]:
#def generate_sonnet(init, model, rounds=600, is_sample=True, temperature=1.0):
#    in_string = init.lower()
#    for i in range(rounds):
#        output = predict_next(string_to_x(in_string), model)
#        if is_sample:
#            n = y_to_char_sample(output, temperature)
#        else:
#            n = y_to_char(output)
#        in_string += n
#    return in_string

In [23]:
def __generate_grammar(model, corpus, abstract_grammars, values, val_indices,
                       indices_val, max_len, max_tries, diversity):
    curr_grammar = ''
    
    # init
    start_index = np.random.randint(0, len(corpus) - max_len)
    sentence = corpus[start_index: start_index + max_len] 
    
    running_length = 0.0
    while running_length <= 4.1:
        x = np.zeros((1, max_len, len(values)))
        for t, val in enumerate(sentence):
            x[0, t, val_indices[val]] = 1.
        
        next_val = __predict(model, x, indices_val, diversity)
        
        if (running_length < 0.00001):
            tries = 0
            while (next_val.split(',')[0] == 'R' or 
                len(next_val.split(',')) != 2):
                
                if tries >= max_tries:
                    print('Gave up on first note generation after', max_tries, 
                        'tries')
                    rand = np.random.randint(0, len(abstract_grammars))
                    next_val = abstract_grammars[rand].split(' ')[0]
                else:
                    next_val = __predict(model, x, indices_val, diversity)
                
                tries += 1
        
        sentence = sentence[1:]  
        sentence.append(next_val)
        
        if (running_length > 0.00001): curr_grammar += ' '
        curr_grammar += next_val
        
        length = float(next_val.split(',')[1])
        running_length += length
    return curr_grammar

In [24]:
max_len = 20
max_tries = 1000
diversity = 1

# musical settings
bpm = 130

In [25]:
# set up audio stream
out_stream = stream.Stream()



In [26]:
curr_offset = 0.0

In [27]:
loopEnd = len(chords)

In [28]:
for loopIndex in range(1, loopEnd):
    curr_chords = stream.Voice()
    for j in chords[loopIndex]:
        curr_chords.insert((j.offset % 4), j)
        
    # generate grammar
    curr_grammar = __generate_grammar(model=model, corpus=corpus, 
                                      abstract_grammars=abstract_grammars, 
                                      values=values, val_indices=val_indices, 
                                      indices_val=indices_val, 
                                      max_len=max_len, max_tries=max_tries,
                                      diversity=diversity)
    
    print(curr_grammar)

    curr_grammar = curr_grammar.replace(' A',' C').replace(' X',' C')

    # Pruning #1: smoothing measure
    curr_grammar = prune_grammar(curr_grammar)

    # Get notes from grammar and chords
    curr_notes = unparse_grammar(curr_grammar, curr_chords)
    
    print(curr_notes)
    
    # Pruning #2: removing repeated and too close together notes
    curr_notes = prune_notes(curr_notes)

    # quality assurance: clean up notes
    curr_notes = clean_up_notes(curr_notes)

    # print # of notes in curr_notes
    print('After pruning: %s notes' % (len([i for i in curr_notes
        if isinstance(i, note.Note)])))
    
    # insert into the output stream
    for m in curr_notes:
        out_stream.insert(curr_offset + m.offset, m)
    
    for mc in curr_chords:
        out_stream.insert(curr_offset + mc.offset, mc)
        
    curr_offset += 4.0

out_stream.insert(0.0, tempo.MetronomeMark(number=bpm))

S,0.250 A,0.250,<m2,P-4> X,0.250,<A4,d-2> C,0.250,<M3,d-3> C,0.250,<M3,d-3> C,0.250,<M3,d-3> C,0.250,<M3,d-3> C,0.250,<P4,m-2> C,0.250,<P4,m-2> C,0.250,<P4,m-2> A,0.250,<M3,d-3> C,0.250,<d5,P1> C,0.250,<d4,M-2> C,0.250,<d4,M-2> C,0.250,<A-4,P-8> C,0.250,<d4,M-2> A,0.250,<m-2,d-6>
<music21.stream.Voice 0x11ad36128>
After pruning: 17 notes
C,0.250 X,0.250,<d1,P-5> C,0.250 C,0.250 S,0.333,<m2,P-4> C,0.250,<P1,d-5> C,0.250,<P5,A1> A,0.250,<M3,d-3> C,0.250,<P5,A1> A,0.250,<M3,d-3> C,0.250,<P5,A1> C,0.250,<A-4,P-8> C,0.250,<d4,M-2> C,0.250,<d4,M-2> C,0.250,<P1,d-5> C,0.250,<P1,d-5> C,0.250,<P1,d-5>
<music21.stream.Voice 0x11c2a3f98>
After pruning: 17 notes
S,0.250 C,0.250,<A-4,P-8> C,0.250,<d4,M-2> C,0.250,<A-4,P-8> S,0.250,<d5,P1> S,0.250,<d5,P1> C,0.250,<A-4,P-8> C,0.250,<P5,A1> C,0.250,<P5,A1> A,0.250,<m-2,d-6> C,0.250,<P5,A1> A,0.250,<m-2,d-6> C,0.250,<P5,A1> A,0.250,<m-2,d-6> A,0.250,<m-2,d-6> C,0.250,<m3,m-3> C,0.250,<d5,P1>
<music21.stream.Voice 0x11ad36860>
After pruning: 16 notes
S,

In [29]:
play = lambda x: midi.realtime.StreamPlayer(x).play()
play(out_stream)

In [30]:
out_fn = './midi/generated1.mid'

In [31]:
mf = midi.translate.streamToMidiFile(out_stream)


In [32]:
mf.open(out_fn, 'wb')
mf.write()
mf.close()